# Current questions / problems

- write searchlight loading
- convert voxel FC to FCMA from brainiak
    - Cython issues...

In [295]:
import glob
import numpy as np
from nilearn import datasets
from nilearn.maskers import MultiNiftiMasker, MultiNiftiLabelsMasker, NiftiSpheresMasker
from nilearn.connectome import ConnectivityMeasure
from sklearn.covariance import EmpiricalCovariance
from scipy.stats import pearsonr

In [142]:
def load_data(FILE_PATHS=[], 
            strategy = 'parcel', 
            schaefer_n_rois=400, 
            sphere_radius=8, 
            sphere_spacing=6):
    """This function loads data in 3 different ways:
        1. Strategy = voxel: extracts direct voxel timeseries
        2. Strategy = parcel: extracts parcel timeseries from schaefer 2018 atlas with a given # of ROIs
        3. Strategy = searchlight: extracts timeseries of spheres of a given radius and spacing

        NOTE: the masker.fit_transform function returns an array of the shape (n_TRs x n_voxels)
        """

    if FILE_PATHS == []:
        FILE_PATHS = glob.glob('data/rest/*.nii.gz') # all rest data by default

    masker_args = {
        'standardize': 'zscore_sample', # ??
        'n_jobs': -1,
        # not doing any: mask_img, smoothing, detrend, standardize, low_pass, high_pass, t_r
    }

    if strategy == 'voxel':
        masker = MultiNiftiMasker(mask_strategy = 'whole-brain-template', # or gm-template?
                                  **masker_args)

    elif strategy == 'parcel':
        schaefer_atlas = datasets.fetch_atlas_schaefer_2018(n_rois=schaefer_n_rois, 
                                                        yeo_networks=7, 
                                                        resolution_mm=2, # because our data is too
                                                        data_dir='data/schaefer', 
                                                        verbose=0)
        masker = MultiNiftiLabelsMasker(labels_img = schaefer_atlas.maps,
                                labels = schaefer_atlas.labels,
                                resampling_target = 'data',
                                strategy = 'mean',
                                **masker_args)

    elif strategy == 'searchlight':
        sphere_coords = [] # get the centerpoint coordinates of spheres. These ranges from chatGPT so def need to check
        for x in range(-90, 91, sphere_spacing):
            for y in range(-126, 91, sphere_spacing):
                for z in range(-72, 73, sphere_spacing):
                    sphere_coords.append((x, y, z))
        masker = NiftiSpheresMasker(seeds = sphere_coords, 
                                    radius=sphere_radius, 
                                    **masker_args)
        # NOTE: change this to match dimensionality of whatever the others are outputting
        return [masker.fit_transform(f) for f in FILE_PATHS]

    # this only works for the multimaskers with voxel/parcel
    return masker.fit_transform(FILE_PATHS)

In [262]:
def compute_fc_voxel(voxel_timeseries, cov_estimator=EmpiricalCovariance()):
    """Full voxel-to-voxel connectivity/correlation matrix
    NOTES:
        - This will take forever with the current implementation -- *replace with FCMA toolbox?*
        - Default nilearn covariance estimator is LedoitWolf, replacing here with EmpiricalCovariance() for pearson
    """
    correlation_measure = ConnectivityMeasure(kind="correlation", cov_estimator=cov_estimator)
    return correlation_measure.fit_transform(voxel_timeseries)

In [314]:
def correlate_rows(mat1, mat2):
    # matrix with the Pearson r correlation of each column (voxel) of mat1 with each column (parcel/target) of mat2
    correlation_matrix = np.empty((mat1.shape[1], mat2.shape[1]))
    for i in range(mat1.shape[0]):
        for j in range(mat2.shape[0]):
            correlation_matrix[i, j] = pearsonr(mat1[:, i], mat2[:, j])[0]
    return correlation_matrix

def compute_fc_target(voxel_timeseries, target_timeseries):
    """Voxel-to-connectivity_target correlation 
    Connectivity targets could be the parcel timeseries, or a searchlight timeseries

    This will take each column in the voxel timeseries (across all the TRs/rows) 
    and correlate it with each column in the target timeseries.

    NOTE: The connectivity target in which a given voxel resides is not excluded yet -- should it be?
    """
    return np.array([correlate_rows(voxel_timeseries[i], target_timeseries[i]) for i in range(len(voxel_timeseries))])

Testing zone

In [313]:
files = glob.glob('data/rest/*.nii.gz')

In [146]:
voxel_data = load_data(FILE_PATHS = [files[0]], strategy='voxel')
parcel_data = load_data(FILE_PATHS = [files[0]], strategy='parcel')

In [296]:
test_fc = compute_fc_voxel(parcel_data, cov_estimator=EmpiricalCovariance())
test_fc[0,0:3,0:3]

c:\Users\ciyer\miniconda3\envs\lab\Lib\site-packages\nilearn\connectome\connectivity_matrices.py:495: FutureWarning: The default strategy for standardize is currently 'zscore' which incorrectly uses population std to calculate sample zscores. The new strategy 'zscore_sample' corrects this behavior by using the sample std. In release 0.13, the default strategy will be replaced by the new strategy and the 'zscore' option will be removed. Please use 'zscore_sample' instead.
  covariances_std = [


array([[1.        , 0.82989885, 0.9214724 ],
       [0.82989885, 1.        , 0.78760728],
       [0.9214724 , 0.78760728, 1.        ]])

In [309]:
test_fc2 = compute_fc_target(voxel_data, parcel_data)
test_fc2[0][0:3, 0:3]

array([[0.78696161, 0.68935141, 0.81799557],
       [0.73036263, 0.58372468, 0.78881791],
       [0.82763582, 0.65915864, 0.84305991]])

In [316]:
file = glob.glob('data/rest/*.nii.gz')[0]
sphere_radius=8, 
sphere_spacing=6

masker_args = {
    'standardize': 'zscore_sample', # ??
    'n_jobs': -1,
    # not doing any: mask_img, smoothing, detrend, standardize, low_pass, high_pass, t_r
}
sphere_coords = [] # get the centerpoint coordinates of spheres. These ranges from chatGPT so def need to check
for x in range(-90, 91, sphere_spacing):
    for y in range(-126, 91, sphere_spacing):
        for z in range(-72, 73, sphere_spacing):
            sphere_coords.append((x, y, z))
masker = NiftiSpheresMasker(seeds = sphere_coords, 
                            radius=sphere_radius, 
                            **masker_args)
# NOTE: change this to match dimensionality of whatever the others are outputting
test = [masker.fit_transform(f) for f in FILE_PATHS]

KeyboardInterrupt: 